In [1]:
# Importando las librerias a utilizar 
import numpy as np

from matplotlib import pyplot

from scipy import optimize

In [2]:
#  datos de entrenamiento almacenados en los arreglos X_train, y_train
data = np.genfromtxt('winequality-red.csv',delimiter=',')
# print(data.shape)
X_train, y_train = data[:, :11], data[:,11]
print(y_train)
y_train = np.array([int(e) for e in y_train])
y_train = np.squeeze(y_train)
# print(y_train)

# # y_train[y_train == 0] = 0
# cambiar las clases de y de 0 a 9
# y_train[y_train == 1] = 0
# y_train[y_train == 2] = 1
# y_train[y_train == 3] = 2
# y_train[y_train == 4] = 3
# y_train[y_train == 5] = 4
# y_train[y_train == 6] = 5
# y_train[y_train == 7] = 6
# y_train[y_train == 8] = 7
# y_train[y_train == 9] = 8
# y_train[y_train == 10] = 9

trainPorcen = int(len(data)/100*90)
testPorcen = int(len(data)/100*10)
print(trainPorcen, testPorcen)

X_test = X_train[trainPorcen: , :11]
y_test = y_train[trainPorcen:]

X_train = X_train[:trainPorcen, :11]
y_train = y_train[:trainPorcen]

print(y_train.shape)
print(y_test.shape)

m = y_train.size

[5. 5. 5. ... 6. 5. 6.]
1439 159
(1439,)
(160,)


In [3]:
# Configurando parametros necesario
input_layer_size  = 11   # Entrada de 11 caracteristicas
hidden_layer_size = 12   # 12 unidades ocultas
num_labels = 10           # 3 etiquetas, de 0 a 2

pesos = {}
pesos['Theta1'] = np.random.rand(12, 12)
pesos['Theta2'] = np.random.rand(10, 13)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

# Desenrollar parámetros
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(144,)
(130,)
(274,)


In [4]:
# # Configurando parametros necesario
# input_layer_size1  = 11   # Entrada de 11 caracteristicas
# hidden_layer_size1 = 14   # 14 unidades ocultas
# input_layer_size2 = 14    # 14 unidades ocultas para la segunda capa
# hidden_layer_size2 = 8    # 5 unidades ocultas
# num_labels = 10           # 3 etiquetas, de 0 a 2

# pesos = {}
# pesos['Theta1'] = np.random.rand(14, 12)
# pesos['Theta2'] = np.random.rand(8, 15)
# pesos['Theta3'] = np.random.rand(10, 9)

# Theta1, Theta2, Theta3= pesos['Theta1'], pesos['Theta2'], pesos['Theta3']

# # Desenrollar parámetros
# print(Theta1.ravel().shape)
# print(Theta2.ravel().shape)
# print(Theta3.ravel().shape)

# nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel(),Theta3.ravel()])
# print(nn_params.shape)

In [5]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z):
  
    g = np.zeros(z.shape)
    g = sigmoid(z) * (1 - sigmoid(z))
    return g

In [6]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_=0.0):
    # pasas de un arreglo plano a un matrix con dimenciones correspondiente    
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size
    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    
    a3 = sigmoid(a2.dot(Theta2.T))
    
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]
    # print(y_matrix)

    temp1 = Theta1
    temp2 = Theta2
    
    # Agregar el termino de regularización
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))
    
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term
    
    # Backpropogation
    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)
    
    # Agregar regularización al gradiente
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    
    return J, grad

In [7]:
# def nnCostFunction(nn_params, input_layer_size1, hidden_layer_size1, input_layer_size2, hidden_layer_size2, num_labels, X, y, lambda_=0.0):
#         pasas de un arreglo plano a un matrix con dimenciones correspondiente   
# #     print(hidden_layer_size1 * (input_layer_size1 + 1), hidden_layer_size2 * (input_layer_size2 + 1))
# #     print(np.reshape(nn_params[:144],()))
    
#     Theta1 = np.reshape(nn_params[:hidden_layer_size1 * (input_layer_size1 + 1)],
#                         (hidden_layer_size1, (input_layer_size1 + 1)))
        
# #     print(Theta1.shape)
# #     print((hidden_layer_size1 * (input_layer_size1 + 1)),hidden_layer_size2 * (input_layer_size2 + 1)+(hidden_layer_size1 * (input_layer_size1 + 1)))
# #     print(hidden_layer_size2, (input_layer_size2 + 1))
    
    
# #     print(np.reshape(nn_params[144:144+78],(6,13)))
# #     Theta2 = np.reshape(nn_params[144:144+78],(6,12))
    
#     Theta2 = np.reshape(nn_params[(hidden_layer_size1 * (input_layer_size1 + 1)):(hidden_layer_size2 * (input_layer_size2 + 1))+(hidden_layer_size1 * (input_layer_size1 + 1))],
#                         (hidden_layer_size2, (input_layer_size2 + 1)))
    
# #     print(Theta2.shape)
    
    
    
# #     print(nn_params[144+78:].shape)
# #     print(num_labels, (hidden_layer_size2 + 1))
    
#     Theta3 = np.reshape(nn_params[(hidden_layer_size2 * (input_layer_size2 + 1))+(hidden_layer_size1 * (input_layer_size1 + 1)):],
#                         (num_labels, (hidden_layer_size2 + 1)))
    
# #     print(Theta3.shape)
    
# #     Theta3 = np.rashape()

#     m = y.size
         
#     J = 0
#     Theta1_grad = np.zeros(Theta1.shape)
#     Theta2_grad = np.zeros(Theta2.shape)
#     Theta3_grad = np.zeros(Theta3.shape)
    
#     a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    
#     a2 = sigmoid(a1.dot(Theta1.T))
#     a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    
#     a3 = sigmoid(a2.dot(Theta2.T))
#     a3 = np.concatenate([np.ones((a3.shape[0], 1)), a3], axis=1)
    
#     a4 = sigmoid(a3.dot(Theta3.T))
    
#     # print("-"*20)
#     # print(y.shape)
#     # print(y.reshape(-1))
#     # print("-"*20)
#     y_matrix = y.reshape(-1)
# #     print(y.reshape(-1))
#     y_matrix = np.eye(num_labels)[y_matrix]
#     # print(y_matrix)

#     temp1 = Theta1
#     temp2 = Theta2
#     temp3 = Theta3
    
#     # Agregar el termino de regularización
    
#     reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:]))+ np.sum(np.square(temp3[:, 1:])))
    
#     J = (-1 / m) * np.sum((np.log(a4) * y_matrix) + np.log(1 - a4) * (1 - y_matrix)) + reg_term
    
#     # Backpropogation
    
#     delta_4 = a4 - y_matrix
#     delta_3 = delta_4.dot(Theta3)[:, 1:] * sigmoidGradient(a2.dot(Theta2.T))
#     delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

#     Delta1 = delta_2.T.dot(a1)
#     Delta2 = delta_3.T.dot(a2)
#     Delta3 = delta_4.T.dot(a3)
    
#     # Agregar regularización al gradiente

#     Theta1_grad = (1 / m) * Delta1
#     Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
#     Theta2_grad = (1 / m) * Delta2
#     Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
    
#     Theta3_grad = (1 / m) * Delta3
#     Theta3_grad[:, 1:] = Theta3_grad[:, 1:] + (lambda_ / m) * Theta3[:, 1:]
# 
#     grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel(), Theta3_grad.ravel()])
    
#     return J, grad

In [8]:
# lambda_ = 0
# J, _ = nnCostFunction(nn_params, input_layer_size1, hidden_layer_size1, input_layer_size2, hidden_layer_size2, num_labels, X_train, y_train, lambda_)
# print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
# print('El costo debe esta cercano a               : 0.287629')

In [9]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X_train, y_train, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
print('El costo debe esta cercano a               : 0.287629')

Costo en parametros (cargado de ex4weights): 59.191490 
El costo debe esta cercano a               : 0.287629


In [10]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [11]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [12]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [13]:
# print('Inicialización de parámetros de redes neuronales...')

# initial_Theta1 = randInitializeWeights(input_layer_size1, hidden_layer_size1)
# initial_Theta2 = randInitializeWeights(input_layer_size2, hidden_layer_size2)
# initial_Theta3 = randInitializeWeights(hidden_layer_size2, num_labels)

# # Desenrrollr parametros
# initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel(), initial_Theta3.ravel()], axis=0)

In [14]:
#  After you have completed the assignment, change the maxiter to a larger
#  value to see how more training helps.
options= {'maxiter': 10000}

#  se puede cambiar diferentes valores
lambda_ = 1

costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X_train, y_train, lambda_)

# Create "short hand" for the cost function to be minimized
# costFunction = lambda p: nnCostFunction(p, input_layer_size1,
#                                         hidden_layer_size1,
#                                         input_layer_size2,
#                                        hidden_layer_size2,
#                                         num_labels, X_train, y_train, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params = res.x

# Thetas para dos capas ocultas
# Theta1 = np.reshape(nn_params[:hidden_layer_size1 * (input_layer_size1 + 1)],
#                         (hidden_layer_size1, (input_layer_size1 + 1)))
    
# Theta2 = np.reshape(nn_params[(hidden_layer_size1 * (input_layer_size1 + 1)):(hidden_layer_size2 * (input_layer_size2 + 1))+(hidden_layer_size1 * (input_layer_size1 + 1))],
#                         (hidden_layer_size2, (input_layer_size2 + 1)))

    
# Theta3 = np.reshape(nn_params[(hidden_layer_size2 * (input_layer_size2 + 1))+(hidden_layer_size1 * (input_layer_size1 + 1)):],
#                         (num_labels, (hidden_layer_size2 + 1)))

    
# Theta para un capa oculta
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

In [15]:
def predict(Theta1, Theta2, X):
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [16]:
# def predict(Theta1, Theta2, Theta3, X):
# 
#     # Useful values
#     m = X.shape[0]
#     num_labels = Theta3.shape[0]

#     # You need to return the following variables correctly
#     p = np.zeros(m)
#     h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
#     h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
#     h3 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h2], axis=1), Theta3.T))
#     p = np.argmax(h3, axis=1)
#     return p

In [17]:
# pred = predict(Theta1, Theta2, Theta3, X_train[:,:])
# print(pred)
# print('Training Set Accuracy: %f' % (np.mean(pred == y_train[:]) * 100))

# pred = predict(Theta1, Theta2, Theta3, X_test[:,:])
# print(pred)
# print('Test Set Accuracy: %f' % (np.mean(pred == y_test[:]) * 100))

In [18]:
pred = predict(Theta1, Theta2, X_train[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y_train[:]) * 100))
pred = predict(Theta1, Theta2, X_test[:,:])
print(pred)
print('Test Set Accuracy: %f' % (np.mean(pred == y_test[:]) * 100))


[5 5 5 ... 5 5 6]
Training Set Accuracy: 64.072272
[6 6 5 6 6 6 5 6 6 5 6 6 6 6 5 6 6 5 5 6 7 6 5 6 6 5 5 5 6 5 5 5 7 6 6 6 7
 6 7 5 6 5 6 5 6 5 5 5 6 5 5 7 5 6 5 6 6 5 6 5 6 5 5 5 6 6 5 5 6 6 6 6 6 6
 5 5 5 6 6 6 5 6 5 6 6 6 6 5 6 5 6 6 5 6 5 6 5 6 6 6 5 7 6 6 6 6 6 6 6 6 6
 5 5 6 5 6 6 5 6 5 5 5 5 5 5 5 6 6 5 5 6 6 6 5 6 6 6 6 6 6 6 6 6 6 5 6 6 6
 6 6 5 6 6 6 5 6 6 6 5 6]
Test Set Accuracy: 58.125000
